Inputs:<br>
  bbb_positive_drugs.csv<br>
  chembl_drug_mechanism_curated.csv<br>
  ad_genes_disgenet.csv<br>
<br>
Outputs:<br>
  phase2_scored_drugs.csv<br>
  phase2_report.txt

In [ ]:
import pandas as pd
import re

In [ ]:
def norm_name(x: str) -> str:
    if pd.isna(x):
        return ""
    x = str(x).lower()
    x = re.sub(r"\(.*?\)", "", x)
    x = re.sub(r"[^a-z0-9\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

In [ ]:
print(" Phase 2 v3 scoring started (pathology-focused)")

--------------------------<br>
1) Load inputs<br>
--------------------------

In [ ]:
bbb = pd.read_csv("../phase1/outputs/bbb_positive_drugs.csv")
moa = pd.read_csv("../database/chembl_drug_mechanism_curated.csv")
ad  = pd.read_csv("../database/ad_genes_disgenet.csv")

--------------------------<br>
2) Detect BBB drug column<br>
--------------------------

In [ ]:
bbb_name_col = None
for c in ["compound_name", "drug_name", "name"]:
    if c in bbb.columns:
        bbb_name_col = c
        break
if bbb_name_col is None:
    raise SystemExit(f" BBB file missing drug name column. Columns: {bbb.columns.tolist()}")

In [ ]:
bbb["drug_norm"] = bbb[bbb_name_col].apply(norm_name)

In [ ]:
if "drug_name" not in moa.columns:
    raise SystemExit(f" chembl_drug_mechanism_curated.csv missing drug_name. Columns: {moa.columns.tolist()}")

In [ ]:
moa["drug_norm"] = moa["drug_name"].apply(norm_name)

Choose best target identifier: gene symbol if present, else target name

In [ ]:
moa["target_gene"] = moa.get("target_gene", "").fillna("").astype(str).str.strip()
moa["target_name"] = moa.get("target_name", "").fillna("").astype(str).str.strip()

In [ ]:
moa["target_best"] = moa["target_gene"]
mask_empty = moa["target_best"].eq("")
moa.loc[mask_empty, "target_best"] = moa.loc[mask_empty, "target_name"]

In [ ]:
moa["t_upper"] = moa["target_best"].astype(str).str.upper()

AD gene set (broad)

In [ ]:
ad_genes_upper = set(ad["gene_symbol"].astype(str).str.strip().str.upper().tolist())

--------------------------<br>
3) Define pathology-focused modules<br>
--------------------------<br>
Core disease-modifying modules

In [ ]:
AMYLOID  = {"APP","BACE1","PSEN1","PSEN2","ADAM10"}
TAU      = {"MAPT","GSK3B","CDK5","MARK4","CSNK1D","CSNK1E"}  # add casein kinases (tau-related)
MICROGLIA= {"TREM2","CSF1R","TYROBP","SPI1"}                  # microglia/immune genetics
LIPID    = {"APOE","CLU","ABCA7","SORL1"}

In [ ]:
CORE = AMYLOID | TAU | MICROGLIA | LIPID

Secondary supportive modules (still relevant, lower weight)

In [ ]:
INFLAM = {"TNF","IL1B","IL6","NFKB1","PTGS2"}
MITO_OX= {"NFE2L2","SOD1","SOD2","PPARGC1A","PINK1","PARK7"}

In [ ]:
SECONDARY = INFLAM | MITO_OX

Symptomatic / nonspecific CNS targets that we DO NOT want to dominate Phase 2<br>
(These are not "wrong", but they aren't disease-modifying signals.)

In [ ]:
EXCLUDE_PREFIXES = (
    "DRD",   # dopamine receptors
    "HTR",   # serotonin receptors
    "ADRA",  # adrenergic receptors
    "CHRM",  # muscarinic receptors
    "GABR",  # GABA receptors
    "OPR",   # opioid receptors
)

Explicitly downweight or exclude a few frequent offenders

In [ ]:
EXCLUDE_EXACT = {
    "NR3C1",  # glucocorticoid receptor: broad stress response
    "CNR1",   # cannabinoid receptor 1
}

Symptomatic Alzheimer target (keep, but low weight)

In [ ]:
LOW_SYMP = {"ACHE"}

In [ ]:
def is_excluded_target(t: str) -> bool:
    t = str(t).upper().strip()
    if t in EXCLUDE_EXACT:
        return True
    return any(t.startswith(p) for p in EXCLUDE_PREFIXES)

In [ ]:
def target_weight(t: str) -> float:
    t = str(t).upper().strip()
    if is_excluded_target(t):
        return 0.0
    if t in CORE:
        return 5.0
    if t in SECONDARY:
        return 2.0
    if t in LOW_SYMP:
        return 0.25

    # Broad AD genes from DisGeNET: very low weight (prevents NR3C1/DRD-like dominance)
    if t in ad_genes_upper:
        return 0.5
    return 0.0

In [ ]:
moa["w"] = moa["t_upper"].apply(target_weight)
moa["is_core_hit"] = moa["t_upper"].isin(CORE)

--------------------------<br>
4) Drug-level features<br>
--------------------------<br>
Total distinct targets in curated MOA set

In [ ]:
num_targets_moa = moa.groupby("drug_norm")["t_upper"].nunique()

Weighted AD score across targets

In [ ]:
moa_w = moa[moa["w"] > 0].copy()
ad_weight_sum = moa_w.groupby("drug_norm")["w"].sum()

Core hits count (STRICT)

In [ ]:
core_hits = moa.groupby("drug_norm")["is_core_hit"].sum()

List of hit targets (only those with w>0)

In [ ]:
ad_hit_targets = moa_w.groupby("drug_norm")["t_upper"].apply(lambda s: ";".join(sorted(set(s))))

In [ ]:
features = pd.DataFrame({
    "drug_norm": num_targets_moa.index,
    "num_targets_moa": num_targets_moa.values
}).merge(
    ad_weight_sum.rename("ad_weight_sum"),
    on="drug_norm",
    how="left"
).merge(
    core_hits.rename("num_core_hits"),
    on="drug_norm",
    how="left"
).merge(
    ad_hit_targets.rename("ad_hit_targets"),
    on="drug_norm",
    how="left"
)

In [ ]:
features["ad_weight_sum"]  = features["ad_weight_sum"].fillna(0.0)
features["num_core_hits"]  = features["num_core_hits"].fillna(0).astype(int)
features["ad_hit_targets"] = features["ad_hit_targets"].fillna("")

--------------------------<br>
5) Merge with BBB list<br>
--------------------------

In [ ]:
out = bbb.merge(features, on="drug_norm", how="left")
out["num_targets_moa"] = out["num_targets_moa"].fillna(0).astype(int)
out["ad_weight_sum"]   = out["ad_weight_sum"].fillna(0.0)
out["num_core_hits"]   = out["num_core_hits"].fillna(0).astype(int)
out["ad_hit_targets"]  = out["ad_hit_targets"].fillna("")
out["drug_name_out"]   = out[bbb_name_col].astype(str)

--------------------------<br>
6) Final scoring rules (pathology-focused)<br>
--------------------------<br>
Normalize by number of targets to avoid promiscuous domination

In [ ]:
out["ad_score_norm"] = out["ad_weight_sum"] / out["num_targets_moa"].clip(lower=1)

Hard requirement: must hit at least 1 core pathology gene to score fully<br>
Otherwise, heavily penalize (still keep a tiny score for secondary-only)

In [ ]:
out["core_gate"] = (out["num_core_hits"] > 0).astype(int)

You can tune these:

In [ ]:
CORE_MULTIPLIER = 1.0
NONCORE_PENALTY = 0.05   # secondary-only gets 5% of score

In [ ]:
out["ad_score_gated"] = out["ad_score_norm"] * (
    out["core_gate"] * CORE_MULTIPLIER + (1 - out["core_gate"]) * NONCORE_PENALTY
)

Optionally include BBB score if you have it

In [ ]:
if "bbb_score" in out.columns:
    out["phase2_score"] = 0.7 * out["ad_score_gated"] + 0.3 * out["bbb_score"].fillna(0)
else:
    out["phase2_score"] = out["ad_score_gated"]

In [ ]:
out = out.sort_values("phase2_score", ascending=False)

--------------------------<br>
7) Save outputs<br>
--------------------------

In [ ]:
out.to_csv("outputs/phase2_scored_drugs.csv", index=False)

In [ ]:
top = out.head(30)[["drug_name_out", "num_targets_moa", "num_core_hits", "ad_hit_targets", "phase2_score"]]

In [ ]:
with open("outputs/phase2_report.txt", "w", encoding="utf-8") as f:
    f.write(f"Total BBB+ drugs: {len(out)}\n")
    nonzero = (out["phase2_score"] > 0).sum()
    f.write(f"Non-zero Phase2 v3 score: {nonzero} ({100*nonzero/len(out):.2f}%)\n")
    f.write(f"Core-hit drugs (num_core_hits>0): {(out['num_core_hits']>0).sum()} ({100*(out['num_core_hits']>0).mean():.2f}%)\n\n")
    f.write("Top 30 candidates:\n")
    f.write(top.to_string(index=False))
    f.write("\n")

In [ ]:
print(" Saved phase2_scored_drugs.csv")
print(" Saved phase2_report.txt")
print("\nTop 30 candidates:")
print(top)